In [1]:
from stark_qa import load_qa, load_skb
from stark_qa.tools.process_text import normalize_answer

In [2]:
dataset_name = 'prime'

# Load the retrieval dataset
qa_dataset = load_qa(dataset_name, human_generated_eval=True)

# Load the semi-structured knowledge base
skb = load_skb(dataset_name, download_processed=False, root='../')

Use file from /Users/louislacombe/.cache/huggingface/hub/datasets--snap-stanford--stark/snapshots/7b0352c7dcefbf254478c203bcfdf284a08866ac/qa/prime/stark_qa/stark_qa_human_generated_eval.csv.
Loading from /Users/louislacombe/CODE/quantmetry/stark/prime/processed!


In [8]:
def check_word_in_text(word, text):
    return word in text

def check_similarity_question_answer(question, list_answers, show=False):
    question_ = normalize_answer(question)
    for answer in list_answers:
        answer_ = normalize_answer(answer)
        if check_word_in_text(answer_, question_):
            if show:
                print('Answer:', answer)
                print('Question:', question)
                print("-"*10)
            return True
    return False


def check_questions(qa_dataset, max_number_answers=5, show=False):
    exclude_questions = []
    for item in qa_dataset:
        question_ = item[0]
        list_answer_ = item[2]

        list_answer_names_ = []
        for answer_ in list_answer_:
            list_answer_names_.append(skb.__getitem__(int(answer_)).name)

        # Check if one of the answers is in the questions using regex
        if check_similarity_question_answer(question_, list_answer_names_, show=show):
            print('Question index:', item[1])
            exclude_questions.append(item[1])

    exclude_questions = list(set(exclude_questions))
    if show:
        print('Number of questions to exclude:', len(exclude_questions))

    qa_dataset_filtered = []
    for item in qa_dataset:
        if item[1] not in exclude_questions:
            qa_dataset_filtered.append(item)
    return qa_dataset_filtered



In [9]:
filtered_questions = check_questions(qa_dataset, show=True)

Answer: mixed mucinous and nonmucinous bronchioloalveolar adenocarcinoma
Question: mixed mucinous and nonmucinous bronchioloalveolar adenocarcinoma is a subtype of what disease?
----------
Question index: 1
Answer: MTND5P11
Question: Is MTND5P11 expressed in any part of the brain?
----------
Question index: 27
Answer: 2,3',4,4',5-pentachlorobiphenyl
Question: What diseases is exposure to 2,3',4,4',5-pentachlorobiphenyl associated with?
----------
Question index: 47
Answer: HIF3A
Question: The protein encoded by HIF3A is associated with negative regulation of what?
----------
Question index: 62
Answer: Protein repair
Question: Complex machine learning methods like alpha fold could help scientists study protein repair and which other pathways?
----------
Question index: 82
Number of questions to exclude: 5
